# Assignment 3
## Question 1: Siamese networks & one-shot learning (8pt)
The Cifar-100 dataset is similar to the Cifar-10 dataset. It also consists of 60,000 32x32 RGB images, but they are distributed over 100 classes instead of 10. Thus, each class has much less examples, only 500 training images and 100 testing images per class. For more info about the dataset, see https://www.cs.toronto.edu/~kriz/cifar.html.

*HINT: Import the Cifar-100 dataset directly from Keras, no need to download it from the website. Use* `label_mode="fine"`

### Task 1.1: Siamese network
**a)**
* Train a Siamese Network on the first 80 classes of (the training set of) Cifar-100, i.e. let the network predict the probability that two input images are from the same class. Use 1 as a target for pairs of images from the same class (positive pairs), and 0 for pairs of images from different classes (negative pairs). Randomly select image pairs from Cifar-100, but make sure you train on as many positive pairs as negative pairs.

* Evaluate the performance of the network on 20-way one-shot learning tasks. Do this by generating 250 random tasks and obtain the average accuracy for each evaluation round. Use the remaining 20 classes that were not used for training. The model should perform better than random guessing.

For this question you may ignore the test set of Cifar-100; it suffices to use only the training set and split this, using the first 80 classes for training and the remaining 20 classes for one-shot testing.

*HINT: First sort the data by their labels (see e.g.* `numpy.argsort()`*), then reshape the data to a shape of* `(n_classes, n_examples, width, height, depth)`*, similar to the Omniglot data in Practical 4. It is then easier to split the data by class, and to sample positive and negative images pairs for training the Siamese network.*

*NOTE: do not expect the one-shot accuracy for Cifar-100 to be similar to that accuracy for Omniglot; a lower accuracy can be expected. However, accuracy higher than random guess is certainly achievable.*

In [0]:
# import libraries
from keras.layers import Input, Conv2D, Lambda, Dense, Flatten, MaxPooling2D, Dropout, BatchNormalization, Dense
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.losses import binary_crossentropy
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.utils import to_categorical
%matplotlib inline

In [0]:
# load data
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode="fine")

In [0]:
y_train_flattened = y_train.flatten()
sort_indices = np.argsort(y_train_flattened, axis=0)
x_train_sorted = x_train[sort_indices]
y_train_sorted = y_train_flattened[sort_indices]

y_test_flattened = y_test.flatten()
sort_indices = np.argsort(y_test_flattened, axis=0)
x_test_sorted = x_test[sort_indices]
y_test_sorted = y_test_flattened[sort_indices]

In [0]:
img_rows, img_cols, chns = 32, 32, 3
n_classes = 100
n_examples = 500
input_shape = (img_rows, img_cols, chns)

x_train_reshaped = np.reshape(x_train_sorted, (n_classes, n_examples, img_rows, img_cols, chns))
x_train_reshaped = x_train_reshaped.astype('float32')
x_train_reshaped /= 255
y_train_reshaped = to_categorical(y_train_sorted, n_classes)

n_examples = 100

x_test_reshaped = np.reshape(x_test_sorted, (n_classes, n_examples, img_rows, img_cols, chns))
x_test_reshaped = x_test_reshaped.astype('float32')
x_test_reshaped /= 255
y_test_reshaped = to_categorical(y_test_sorted, n_classes)

In [13]:
print(x_train_sorted.shape)
print(y_train_reshaped.shape)
print(x_train_reshaped.shape)
print(x_train_reshaped[:80].shape)
print(x_train_reshaped[80:].shape)

(50000, 32, 32, 3)
(50000, 100)
(100, 500, 32, 32, 3)
(80, 500, 32, 32, 3)
(20, 500, 32, 32, 3)


In [25]:
left_input = Input(input_shape)
right_input = Input(input_shape)

# build convnet to use in each siamese 'leg'
convnet = Sequential()
convnet.add(Conv2D(64, (4,4), activation='relu', input_shape=input_shape, kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(128, (3,3), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(128, (2,2), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(MaxPooling2D())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Conv2D(256, (2,2), activation='relu', kernel_regularizer=l2(2e-4)))
convnet.add(Flatten())
convnet.add(BatchNormalization())
convnet.add(Dropout(0.25))
convnet.add(Dense(4096, activation="sigmoid", kernel_regularizer=l2(1e-3)))
convnet.summary()

# encode each of the two inputs into a vector with the convnet
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)

# merge two encoded inputs with the L1 distance between them, and connect to prediction output layer
L1_distance = lambda x: K.abs(x[0]-x[1])
both = Lambda(L1_distance)([encoded_l, encoded_r])
prediction = Dense(1, activation='sigmoid')(both)
siamese_net = Model(inputs=[left_input,right_input], outputs=prediction)


siamese_net.compile(loss="binary_crossentropy", optimizer="adam")

siamese_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 29, 29, 64)        3136      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_27 (Batc (None, 14, 14, 64)        256       
_________________________________________________________________
dropout_27 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
batch_normalization_28 (Batc (None, 6, 6, 128)         512       
__________

In [0]:
def get_batch(batch_size, X):
    """Create batch of n pairs, half same class, half different class"""
    n_classes, n_examples, w, h, d = X.shape
    # randomly sample several classes to use in the batch
    categories = np.random.choice(n_classes, size=(batch_size,), replace=False)
    # initialize 2 empty arrays for the input image batch
    pairs = [np.zeros((batch_size, h, w, d)) for i in range(2)]
    # initialize vector for the targets, and make one half of it '1's, so 2nd half of batch has same class
    targets = np.zeros((batch_size,))
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = categories[i]
        idx_1 = np.random.randint(0, n_examples)
        pairs[0][i, :, :, :] = X[category, idx_1].reshape(w, h, d)
        idx_2 = np.random.randint(0, n_examples)
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category
        else:
            #add a random number to the category modulo n_classes to ensure 2nd image has different category
            category_2 = (category + np.random.randint(1,n_classes)) % n_classes
        pairs[1][i, :, :, :] = X[category_2,idx_2].reshape(w, h, d)
    return pairs, targets

def batch_generator(batch_size, X):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batch(batch_size, X)
        yield (pairs, targets)

def train(model, X, batch_size=64, steps_per_epoch=100, epochs=1):
    model.fit_generator(batch_generator(batch_size, X), steps_per_epoch=steps_per_epoch, epochs=epochs)

In [0]:
def make_oneshot_task(N, X):
    """Create pairs of (test image, support set image) with ground truth, for testing N-way one-shot learning."""
    n_classes, n_examples, w, h, d = X.shape
    indices = np.random.randint(0, n_examples, size=(N,))
    
    # pick a bunch of random labels
    categories = np.random.choice(range(n_classes), size=(N,), replace=False)            
    
    true_category = categories[0]
    ex1, ex2 = np.random.choice(n_examples, replace=False, size=(2,))
    test_image = np.asarray([X[true_category, ex1, :, :]]*N).reshape(N, w, h, d)
    support_set = X[categories, indices, :, :]
    support_set[0, :, :] = X[true_category, ex2]
    support_set = support_set.reshape(N, w, h, d)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image, support_set]
    return pairs, targets

def test_oneshot(model, X, N=20, k=250, verbose=True):
    """Test average N-way oneshot learning accuracy of a siamese neural net over k one-shot tasks."""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {}-way one-shot learning tasks ...".format(k, N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N, X)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct += 1
    percent_correct = (100.0*n_correct / k)
    if verbose:
        print("Got an average of {}% accuracy for {}-way one-shot learning".format(percent_correct, N))
    return percent_correct

In [27]:
loops = 100
best_acc = 0
worst_acc = 100
av_acc = 0

for i in range(loops):
    print("\n=== Training loop {} ===".format(i+1))
    train(siamese_net, x_train_reshaped[:80])
    test_acc = test_oneshot(siamese_net, x_train_reshaped[80:])
    av_acc = av_acc + test_acc
    if test_acc >= best_acc:
        print("New best one-shot accuracy: " + str(test_acc))
        best_acc = test_acc
    if test_acc < worst_acc:
        print("New worst one-shot accuracy: " + str(test_acc))
        worst_acc = test_acc
        
av_acc = av_acc/loops
print("Average one-shot accuracy: " + str(av_acc))
print("Final best one-shot accuracy: " + str(best_acc))
print("Final worst one-shot accuracy: " + str(worst_acc))


=== Training loop 1 ===
Epoch 1/1
100/100 [==============================] - 6s 57ms/step - loss: 0.6906
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 8.8% accuracy for 20-way one-shot learning
New best one-shot accuracy: 8.8
New worst one-shot accuracy: 8.8

=== Training loop 2 ===
Epoch 1/1
100/100 [==============================] - 6s 55ms/step - loss: 0.6839
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.0% accuracy for 20-way one-shot learning
New best one-shot accuracy: 12.0

=== Training loop 3 ===
Epoch 1/1
 77/100 [======================>.......] - ETA: 1s - loss: 0.6812

100/100 [==============================] - 6s 56ms/step - loss: 0.6845
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.4% accuracy for 20-way one-shot learning
New best one-shot accuracy: 12.4

=== Training loop 4 ===
Epoch 1/1
100/100 [==============================] - 6s 55ms/step - loss: 0.6876
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 13.6% accuracy for 20-way one-shot learning
New best one-shot accuracy: 13.6

=== Training loop 5 ===
Epoch 1/1
100/100 [==============================] - 5s 52ms/step - loss: 0.6855
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 11.2% accuracy for 20-way one-shot learning

=== Training loop 6 ===
Epoch 1/1
 25/100 [======>.......................] - ETA: 3s - loss: 0.6810

100/100 [==============================] - 5s 51ms/step - loss: 0.6809
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.0% accuracy for 20-way one-shot learning

=== Training loop 7 ===
Epoch 1/1
100/100 [==============================] - 5s 50ms/step - loss: 0.6763
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 13.2% accuracy for 20-way one-shot learning

=== Training loop 8 ===
Epoch 1/1
100/100 [==============================] - 5s 50ms/step - loss: 0.6852
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 14.4% accuracy for 20-way one-shot learning
New best one-shot accuracy: 14.4

=== Training loop 9 ===
Epoch 1/1
 60/100 [=================>............] - ETA: 1s - loss: 0.6799

100/100 [==============================] - 5s 49ms/step - loss: 0.6813
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 11.2% accuracy for 20-way one-shot learning

=== Training loop 10 ===
Epoch 1/1
100/100 [==============================] - 5s 49ms/step - loss: 0.6851
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 11.2% accuracy for 20-way one-shot learning

=== Training loop 11 ===
Epoch 1/1
100/100 [==============================] - 5s 50ms/step - loss: 0.6911
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.4% accuracy for 20-way one-shot learning

=== Training loop 12 ===
Epoch 1/1
 73/100 [====================>.........] - ETA: 1s - loss: 0.6937

100/100 [==============================] - 5s 48ms/step - loss: 0.6906
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.0% accuracy for 20-way one-shot learning

=== Training loop 13 ===
Epoch 1/1
100/100 [==============================] - 5s 49ms/step - loss: 0.6929
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 18.0% accuracy for 20-way one-shot learning
New best one-shot accuracy: 18.0

=== Training loop 14 ===
Epoch 1/1
100/100 [==============================] - 5s 49ms/step - loss: 0.6814
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.0% accuracy for 20-way one-shot learning

=== Training loop 15 ===
Epoch 1/1
 64/100 [==================>...........] - ETA: 1s - loss: 0.6857

...

100/100 [==============================] - 5s 49ms/step - loss: 0.6704
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 12.8% accuracy for 20-way one-shot learning

=== Training loop 88 ===
Epoch 1/1
100/100 [==============================] - 5s 48ms/step - loss: 0.6661
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 13.2% accuracy for 20-way one-shot learning

=== Training loop 89 ===
Epoch 1/1
100/100 [==============================] - 5s 48ms/step - loss: 0.6639
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 13.2% accuracy for 20-way one-shot learning

=== Training loop 90 ===
Epoch 1/1
 85/100 [========================>.....] - ETA: 0s - loss: 0.6762

100/100 [==============================] - 5s 48ms/step - loss: 0.6779
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 15.2% accuracy for 20-way one-shot learning

=== Training loop 91 ===
Epoch 1/1
100/100 [==============================] - 5s 48ms/step - loss: 0.6691
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 10.4% accuracy for 20-way one-shot learning

=== Training loop 92 ===
Epoch 1/1
100/100 [==============================] - 5s 48ms/step - loss: 0.6686
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 20.8% accuracy for 20-way one-shot learning
New best one-shot accuracy: 20.8

=== Training loop 93 ===
Epoch 1/1
 97/100 [============================>.] - ETA: 0s - loss: 0.6746

100/100 [==============================] - 5s 48ms/step - loss: 0.6746
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 16.0% accuracy for 20-way one-shot learning

=== Training loop 94 ===
Epoch 1/1
100/100 [==============================] - 5s 47ms/step - loss: 0.6748
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 15.6% accuracy for 20-way one-shot learning

=== Training loop 95 ===
Epoch 1/1
100/100 [==============================] - 5s 48ms/step - loss: 0.6681
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 10.0% accuracy for 20-way one-shot learning

=== Training loop 96 ===
Epoch 1/1
 99/100 [============================>.] - ETA: 0s - loss: 0.6724

100/100 [==============================] - 5s 48ms/step - loss: 0.6722
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 16.0% accuracy for 20-way one-shot learning

=== Training loop 97 ===
Epoch 1/1
100/100 [==============================] - 5s 48ms/step - loss: 0.6729
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 15.6% accuracy for 20-way one-shot learning

=== Training loop 98 ===
Epoch 1/1
100/100 [==============================] - 5s 48ms/step - loss: 0.6644
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 13.6% accuracy for 20-way one-shot learning

=== Training loop 99 ===
Epoch 1/1
 66/100 [==================>...........] - ETA: 1s - loss: 0.6651

100/100 [==============================] - 5s 48ms/step - loss: 0.6622
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 14.0% accuracy for 20-way one-shot learning

=== Training loop 100 ===
Epoch 1/1
100/100 [==============================] - 5s 48ms/step - loss: 0.6587
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 18.4% accuracy for 20-way one-shot learning
Average one-shot accuracy: 14.039999999999994
Final best one-shot accuracy: 20.8
Final worst one-shot accuracy: 8.8


1***

**b)** Briefly motivate your model's architecture, as well as its performance. What accuracy would random guessing achieve (on average)?

**Answer:**

Code based on: https://sorenbouma.github.io/blog/oneshot/ and https://github.com/sorenbouma/keras-oneshot/blob/master/SiameseNet.ipynb, similar to the practical we included dropout and batch normalization to improve generalization and speed up training.

It was adapted to have a smaller kernel size at all the different convolutional layers, because the input images are smaller than the Omniglot data.

The convolutional layers were set to 4x4, 3x3, 2x2, 2x2 for the first, second, third, and fourth layer respectively initially, this already produced results well above random guessing (random guessing would result in a 5% accuracy) with an accuracy around 20%.

After it was tried to increase the first convolutional layer to 5x5, increase the first convolutional layer to 5x5 and remove a MaxPool layer, and completely remove the 256 dimensional convolutional layer. All of these efforts resulted in a lower accuracy by a number of percentages, so the model was returned to its initial and final form.

Average one-shot accuracy: 14.039999999999994

Final best one-shot accuracy: 20.8

Final worst one-shot accuracy: 8.8

***

**c)** Compare the performance of your Siamese network for Cifar-100 to the Siamese network from Practical 4 for Omniglot. Name three fundamental differences between the Cifar-100 and Omniglot datasets. How do these differences influence the difference in one-shot accuracy?

**Answer:**

*=== write your answer here ===*

Omniglot has pictures of 105 by 105, higher quality than CIFAR-100 data, which has 32 by 32 images.

Omniglot has a depth of 1, while the depth of CIFAR-100 has 3 (RGB) 



***

### Task 1.2: One-shot learning with neural codes
**a)**
* Train a CNN classifier on the first 80 classes of Cifar-100. Make sure it achieves at least 40% classification accuracy on those 80 classes (use the test set to validate this accuracy).
* Then use neural codes from one of the later hidden layers of the CNN with L2-distance to evaluate one-shot learning accuracy for the remaining 20 classes of Cifar-100. I.e. for a given one-shot task, obtain neural codes for the test image as well as the support set. Then pick the image from the support set that is closest (in L2-distance) to the test image as your one-shot prediction.

In [21]:
print(x_train_sorted.shape)
print(y_train_reshaped.shape)

print(x_test_sorted.shape)
print(y_test_reshaped.shape)

print("example one-hot encoding:", y_train_reshaped[0])

(50000, 32, 32, 3)
(50000, 100)
(10000, 32, 32, 3)
(10000, 100)
example one-hot encoding: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [22]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(256, activation='relu', name="neural_codes"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(n_classes, activation='softmax'))

model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
dropout_6 (Dropout)          (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
batch_normalization_7 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
dropout_7 (Dropout)          (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 26, 26, 64)        18496     
__________

In [24]:
batch_size = 100
epochs = 20

model.fit(x_train_sorted[:40000], y_train_reshaped[:40000],
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1)

Train on 36000 samples, validate on 4000 samples
Epoch 1/20
36000/36000 [==============================] - 31s 863us/step - loss: 3.3204 - acc: 0.2217 - val_loss: 7.1558 - val_acc: 0.0000e+00
Epoch 2/20
16700/36000 [============>.................] - ETA: 14s - loss: 2.4374 - acc: 0.3745

36000/36000 [==============================] - 29s 798us/step - loss: 2.3850 - acc: 0.3830 - val_loss: 8.1239 - val_acc: 0.0000e+00
Epoch 3/20
36000/36000 [==============================] - 29s 800us/step - loss: 1.9233 - acc: 0.4836 - val_loss: 9.1782 - val_acc: 0.0000e+00
Epoch 4/20
  500/36000 [..............................] - ETA: 28s - loss: 1.4665 - acc: 0.6040

36000/36000 [==============================] - 29s 796us/step - loss: 1.5555 - acc: 0.5755 - val_loss: 9.5480 - val_acc: 0.0000e+00
Epoch 5/20
30900/36000 [========================>.....] - ETA: 3s - loss: 1.1848 - acc: 0.6660

36000/36000 [==============================] - 29s 794us/step - loss: 1.2043 - acc: 0.6593 - val_loss: 10.6269 - val_acc: 0.0000e+00
Epoch 6/20
36000/36000 [==============================] - 29s 794us/step - loss: 0.8682 - acc: 0.7546 - val_loss: 11.2377 - val_acc: 0.0000e+00
Epoch 7/20
 3600/36000 [==>...........................] - ETA: 25s - loss: 0.5254 - acc: 0.8586

36000/36000 [==============================] - 29s 799us/step - loss: 0.6071 - acc: 0.8261 - val_loss: 11.5075 - val_acc: 0.0000e+00
Epoch 8/20
31900/36000 [=========================>....] - ETA: 3s - loss: 0.4216 - acc: 0.8775

...

36000/36000 [==============================] - 29s 795us/step - loss: 0.1831 - acc: 0.9428 - val_loss: 13.7568 - val_acc: 0.0000e+00
Epoch 15/20
36000/36000 [==============================] - 29s 796us/step - loss: 0.1716 - acc: 0.9450 - val_loss: 14.1842 - val_acc: 0.0000e+00
Epoch 16/20
 3700/36000 [==>...........................] - ETA: 25s - loss: 0.1389 - acc: 0.9551

36000/36000 [==============================] - 29s 795us/step - loss: 0.1644 - acc: 0.9479 - val_loss: 14.2157 - val_acc: 0.0000e+00
Epoch 17/20
31800/36000 [=========================>....] - ETA: 3s - loss: 0.1499 - acc: 0.9524

36000/36000 [==============================] - 29s 795us/step - loss: 0.1542 - acc: 0.9509 - val_loss: 14.0692 - val_acc: 0.0000e+00
Epoch 18/20
36000/36000 [==============================] - 29s 794us/step - loss: 0.1484 - acc: 0.9523 - val_loss: 14.0389 - val_acc: 0.0000e+00
Epoch 19/20
 3700/36000 [==>...........................] - ETA: 24s - loss: 0.1303 - acc: 0.9565

36000/36000 [==============================] - 29s 796us/step - loss: 0.1380 - acc: 0.9566 - val_loss: 14.5010 - val_acc: 0.0000e+00
Epoch 20/20
31800/36000 [=========================>....] - ETA: 3s - loss: 0.1227 - acc: 0.9595

36000/36000 [==============================] - 29s 794us/step - loss: 0.1249 - acc: 0.9588 - val_loss: 14.6845 - val_acc: 0.0000e+00


In [26]:
loss, accuracy = model.evaluate(x_test_sorted[:8000], y_test_reshaped[:8000], verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 4.397154795885086
Test accuracy: 0.417375


Test loss: 2.4939483386993406
Test accuracy: 0.3731

dropout 025: 
Test loss: 3.3200662326812744
Test accuracy: 0.3849

dropout 0.25+extra 16 layer: 
Test loss: 3.1162593254089357
Test accuracy: 0.3892

dropout 0.2 + extra 16 layer:
Test loss: 4.177343496441841
Test accuracy: 0.398

same (32, 32, 64, 128, 256):
Test loss: 4.397154795885086
Test accuracy: 0.417375

Then use neural codes from one of the later hidden layers of the CNN with L2-distance to evaluate one-shot learning accuracy for the remaining 20 classes of Cifar-100. I.e. for a given one-shot task, obtain neural codes for the test image as well as the support set. Then pick the image from the support set that is closest (in L2-distance) to the test image as your one-shot prediction.

def test_oneshot

L2 met nearest neighbor toepassen



In [0]:
from sklearn.neighbors import NearestNeighbors
n_classes = 100

def make_oneshot_task_cnn(N, X):
    """Create pairs of (test image, support set image) with ground truth, for testing N-way one-shot learning."""
    n_examples, w, h, d = X.shape
    indices = np.random.randint(0, n_examples, size=(N,))
    
    # pick a bunch of random labels
    categories = np.random.choice(range(n_classes), size=(N,), replace=False)            
    
    true_category = categories[0]
    ex1, ex2 = np.random.choice(n_examples, replace=False, size=(2,))
    test_image = np.asarray([X[ex1, :, :]]*N).reshape(N, w, h, d)
    support_set = X[indices, :, :]
    support_set[0, :, :] = X[ex2]
    support_set = support_set.reshape(N, w, h, d)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image, support_set]
    return pairs, targets
  
def test_oneshot_cnn(model, X, N=20, k=250, verbose=True):
    """Test average N-way oneshot learning accuracy of a convolutional neural net over k one-shot tasks."""
    n_correct = 0
    neigh = NearestNeighbors(n_neighbors=1, p=2)
    if verbose:
        print("Evaluating model on {} random {}-way one-shot learning tasks ...".format(k, N))
    for i in range(k):
        inputs, targets = make_oneshot_task_cnn(N, X)
        test_image = np.array([inputs[0][0]])
        support_set = inputs[1]
        test_image_code = model.predict(test_image)
        support_set_codes = model.predict(support_set)
        
        neigh.fit(support_set_codes)
        nn = neigh.kneighbors(test_image_code)
        
        nn_index = nn[1][0][0]
        if nn_index == np.argmax(targets):
          n_correct += 1
    percent_correct = (100.0*n_correct / k)
    if verbose:
        print("Got an average of {}% accuracy for {}-way one-shot learning".format(percent_correct, N))
    return percent_correct

In [39]:
neural_codes = Model(inputs=model.input, outputs=model.get_layer("neural_codes").output)

loops = 100
best_acc = 0
worst_acc = 100
av_acc = 0

for i in range(loops):
    print("\n=== Training loop {} ===".format(i+1))
    #train(siamese_net, x_train_reshaped[:80])
    test_acc = test_oneshot_cnn(neural_codes, x_train_sorted[40000:])
    av_acc = av_acc + test_acc
    if test_acc >= best_acc:
        print("New best one-shot accuracy: " + str(test_acc))
        best_acc = test_acc
    if test_acc < worst_acc:
        print("New worst one-shot accuracy: " + str(test_acc))
        worst_acc = test_acc
        
av_acc = av_acc/loops

print("Average one-shot accuracy: " + str(av_acc))
print("Final best one-shot accuracy: " + str(best_acc))
print("Final worst one-shot accuracy: " + str(worst_acc))


=== Training loop 1 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 6.4% accuracy for 20-way one-shot learning
New best one-shot accuracy: 6.4
New worst one-shot accuracy: 6.4

=== Training loop 2 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 5.2% accuracy for 20-way one-shot learning
New worst one-shot accuracy: 5.2

=== Training loop 3 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 2.0% accuracy for 20-way one-shot learning
New worst one-shot accuracy: 2.0

=== Training loop 4 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 5.2% accuracy for 20-way one-shot learning

=== Training loop 5 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 2.8% accuracy for 20-way one-shot learning

=== Training loop 6 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 

Got an average of 4.4% accuracy for 20-way one-shot learning

=== Training loop 36 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 6.8% accuracy for 20-way one-shot learning

=== Training loop 37 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 3.2% accuracy for 20-way one-shot learning

=== Training loop 38 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 4.4% accuracy for 20-way one-shot learning

=== Training loop 39 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 4.8% accuracy for 20-way one-shot learning

=== Training loop 40 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 4.8% accuracy for 20-way one-shot learning

=== Training loop 41 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 6.4% accuracy for 20-way one-shot learning

=== Training loop 42

Got an average of 4.8% accuracy for 20-way one-shot learning

=== Training loop 71 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 4.4% accuracy for 20-way one-shot learning

=== Training loop 72 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 4.8% accuracy for 20-way one-shot learning

=== Training loop 73 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 6.8% accuracy for 20-way one-shot learning

=== Training loop 74 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 5.2% accuracy for 20-way one-shot learning

=== Training loop 75 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 6.8% accuracy for 20-way one-shot learning

=== Training loop 76 ===
Evaluating model on 250 random 20-way one-shot learning tasks ...
Got an average of 4.4% accuracy for 20-way one-shot learning

=== Training loop 77

***

**b)** Briefly motivate your CNN architecture, and discuss the difference in one-shot accuracy between the Siamese network approach and the CNN neural codes approach.

**Answer:**

*=== write your answer here ===*

Average one-shot accuracy: 5.0920000000000005
Final best one-shot accuracy: 10.0
Final worst one-shot accuracy: 2.0